# Load data and libraries

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install shap
!pip install pyitlib

In [0]:
import os
os.path.abspath(os.getcwd())
os.chdir('/content/drive/My Drive/Protein project')
os.path.abspath(os.getcwd())

from __future__ import division ###for float operation
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score ##tp / (tp + fn)
from sklearn.metrics import precision_score #tp / (tp + fp)
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold, StratifiedKFold
from pyitlib import discrete_random_variable as drv
import time 
import timeit 


def readData(filename):
	fr = open(filename)
	returnData = []
	headerLine=fr.readline()###move cursor 
	for line in fr.readlines():
		lineStrip = line.strip().replace('"','')
		lineList =	lineStrip.split('\t')
		returnData.append(lineList)###['3','2',...]
	return returnData

"""first case P450 = [['1','1',....],[],[].....,[]]  second case P450 = array([['1','1',....],[],[].....,[]]), third case P450 = """
P450 = readData('P450.txt') ### [[],[],[],....[]]
P450 = np.array(P450) ### either [['1','1',....],[],[].....,[]] or array([['1','1',....],[],[].....,[]]) works, but note that keys are '1', '0' 
#P450 = P450.astype(int) ### for shap array [[1,1,....],[],[].....,[]], keys are 1, 0 

M=np.matrix([[245,   9,   0,   3,   0,   2,  65,   8],
[9, 218,  17,  17,  49,  10,  50,  17],
[0,  17, 175,  16,  25,  13,   0,  46],
[3,  17,  16, 194,  19,   0,   0,   3],
[0,  49,  25,  19, 199,  10,   0,   3],
[2,  10,  13,   0,  10, 249,  50,  74],
[65,  50,   0,   0,   0,  50, 262,  11],
[8,  17,  46,   3,   3,  74,  11, 175]])

X = P450[:,0:8]
y = P450[:,-1]

In [0]:
def readData2(filename):
	fr = open(filename)
	returnData = []
	headerLine=fr.readline()###move cursor 
	for line in fr.readlines():
		linestr = line.strip().replace(', ','')
		lineList =	list(linestr)
		returnData.append(lineList)###['3','2',...]
	return returnData

lactamase = readData2('lactamase.txt')
lactamase = np.array(lactamase)
#lactamase = lactamase.astype(int)
M2 = np.matrix([[101, 5, 0, 2, 0, 14, 4, 37],
[5 ,15, 14 ,1 ,7 ,7, 0 ,19],
[0, 14, 266, 15, 14, 2, 26, 4],
[2, 1, 15, 28, 2 ,15, 4, 0],
[0, 7, 14, 2, 32, 9 ,0, 8],
[14, 7, 2 ,15, 9, 29, 7, 9],
[4, 0, 26, 4 ,0 ,7 ,72, 21],
[37, 19, 4, 0, 8, 9, 21, 211]])

X2 = lactamase[:,0:8]
y2 = lactamase[:,-1]

In [0]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted ### Checks if the estimator is fitted by verifying the presence of fitted attributes (ending with a trailing underscore)
#from sklearn.utils.multiclass import unique_labels, not necessary, can be replaced by array(list(set()))

# Bayesian network

In [0]:
"""
Bayesian network implementation
API inspired by SciKit-learn.
"""

class Bayes_net(BaseEstimator, ClassifierMixin): 
    
    def fit(self,X,y,M = None):
      raise NotImplementedError

    def predict_proba(self, X): ### key prediction methods, all other prediction methods will use it first.
      raise NotImplementedError

    def predict_binary(self,X):
      """
        Perform classification on an array of test vectors X, predict P(C1|X), works only for binary classifcation
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        C : ndarray of shape (n_samples,)
            Predicted P(C1|X)
      """
      Prob_C = self.predict_proba(X) ### Prob_C is n*|C| np.array
      return(Prob_C[:,0]) 

    def predict(self, X):
      """
        Perform classification on an array of test vectors X.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        C : ndarray of shape (n_samples,)
            Predicted target values for X
      """
      Prob_C = self.predict_proba(X) ## Prob_C is |C|*n np.array ,C is self.C 
      return( np.array([self.classes_[ele] for ele in np.argmax(Prob_C, axis=1)] ) )

    def Conditional_log_likelihood_general(self,y_true,y_pred_prob,C): 
      """Calculate the conditional log likelihood.
      :param y_true: The true class labels. e.g ['1','1',.....'0','0']
      :param y_pred_prob: np.array shows prob of each class for each instance. ith column is the predicted prob for class C[i]
      :param C: Class labels  e.x array(['1','0']), C has to use same labels as y_true.
      :return: CLL. A scalar.
      """
      C = list(C) ## only list can use .index
      cll = []
      for i in range(len(y_true)):
        cll.append( y_pred_prob[i,C.index(y_true[i])] ) ## \hat p(c_true|c_true)
      
      cll = [np.log2(ele) for ele in cll]
      cll = np.array(cll)
      return(sum(cll))
 

## Naive Bayes

In [0]:
class NB(Bayes_net):
  name = "NB"
  def __init__(self, alpha = 1):
      self.alpha = alpha

  def fit(self,X, y, M = None):  
    """ Implementation of a fitting function.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            The training input samples.
        y : array-like, shape (n_samples,) or (n_samples, n_outputs)
            The target values (class labels in classification, real numbers in
            regression).
        Returns
        -------
        self : object
            Returns self.
      """

      #  countDict_, classes_, p_ , P_class_prior_, Dict_C_, K_ ,training_time_, is_fitted_  are fitted "coef_" 
       # coef_ has to been refreshed each fitting. 

    X, y = check_X_y(X, y)
    t = time.process_time()
    #start timing
    countDict = Counter(y) ## {c1:n1,c2:n2,c3:n3} sorted by counts
    C = list(countDict.keys()) ### [class1 , class2, class3] in appearing order
    n,p = X.shape   ## num of features 8                                                           ### values same order as .keys()
    P_class_prior = [(ele+self.alpha)/ ( n + self.alpha*len(C) )  for ele in countDict.values()]  ### prior for each class [p1,p2,p3]
    P_class_prior = dict(zip(C, P_class_prior))  ## {c1:p1,c2:p2,c3:p3} ## should in correct order, .keys .values.
    Dict_C = {} ###  {c1:[counter1, ....counter8], c2:[counter1, ....counter8],   c3: [counter1, ....counter8]}
    K = {} ## [x1 unique , x2 unique .... x8unique]

    for c in C:
      ListCounter_c = []

      for i in range(p):
        row_inx_c = [row for row in range(n) if y[row] == c]
        x_i_c = X[row_inx_c,i]
        ListCounter_c.append(Counter(x_i_c))
        if c == C[0]:
          x_i = X[:,i]
          K[i] = len(Counter(x_i))

      Dict_C[c] = ListCounter_c
    
    CP_time = np.array(time.process_time() - t) 
    self.is_fitted_ = True    
    self.Dict_C_,self.p_,self.P_class_prior_,self.K_,self.classes_,self.countDict_,self.training_time_ = Dict_C,p,P_class_prior,K,np.array(C),countDict,CP_time
    return self


  def predict_proba(self,X): 
    """
        Return probability estimates for the test vector X.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        Returns
        -------
        C : array-like of shape (n_samples, n_classes)
            Returns the probability of the samples for each class in
            the model. The columns correspond to the classes in sorted
            order, as they appear in the attribute :term:`classes_`.
    """
    check_is_fitted(self)
    X = check_array(X)
    Prob_C = []
    for ins in X:
      P_class = self.P_class_prior_.copy() ### {c1:p1, c2:p2} #### !!!! dict1 = dict2 , change both simultaneously!!!
      for c in self.classes_:
        ListCounter_c = self.Dict_C_[c]
        for i in range(self.p_):
          P_class[c] = P_class[c] * (ListCounter_c[i][ins[i]]+self.alpha) / (self.countDict_[c] + self.alpha*self.K_[i])
        
      ## normalize P_class
      P_class = {key: P_class[key]/sum(list(P_class.values())) for key in P_class.keys()}
      Prob_C.append(list(P_class.values())) ### check the class order is correct
  
    Prob_C = np.array(Prob_C) ### for shap !!!!
    return Prob_C

    

In [38]:
nb = NB()
nb.fit(X,y)
nb.predict_proba(X)
#nb.get_params()
#nb.classes_
print(nb.name)
print(nb.predict_proba(X))
nb.score(X,y)

NB
[[0.21028538 0.78971462]
 [0.83483797 0.16516203]
 [0.72336329 0.27663671]
 ...
 [0.42722113 0.57277887]
 [0.4165342  0.5834658 ]
 [0.62970704 0.37029296]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:755: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  estimator=estimator)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in sciki

0.8104712041884817

## TAN

In [0]:
class TAN(Bayes_net):
    name = "TAN"
    def __init__(self, alpha = 1,starting_node = 0):
      self.starting_node = starting_node
      self.alpha = alpha

    def To_CAT(self, X_i): 
      """For using CMI purpose, convert X_i e.g ['a','b','a']/['0','1','0']  to [0,1,0].
      :param X_i: one feature column. 
      :return: list(type int)
      """
      X_i_list = list(set(X_i));X_i_dict = dict(zip(X_i_list, range(len(X_i_list)) ))
      return([X_i_dict[ele] for ele in X_i])

    def get_mutual_inf(self,X,Y):
      """get conditional mutual inf of all pairs of features, part of training
      :return: np.array matrix.
      """
      t = time.process_time()
      n,p = X.shape 
      M = np.zeros((p,p))
      Y = self.To_CAT(Y)
      for i in range(p):
        X_i = X[:,i]
        X_i = self.To_CAT(X_i)
        for j in range(p):
          X_j = X[:,j] 
          X_j = self.To_CAT(X_j)
          M[i,j] = drv.information_mutual_conditional(X_i,X_j,Y)
      
      mutual_inf_time = time.process_time() - t
      return M, mutual_inf_time

    def Findparent(self,X,Y):
      M,mutual_inf_time = self.get_mutual_inf(X,Y)
      t = time.process_time()
      np.fill_diagonal(M,0)  
      p = int(M.shape[0])  
      V = range(p)  #### . set of all nodes
      st = self.starting_node
      Vnew = [st] #### vertex that already found their parent. intitiate it with starting node. TAN randomly choose one
      parent = {st:None} ## use a dict to show nodes' interdepedency
      while set(Vnew) != set(V):   ### when their are still nodes whose parents are unknown.
        index_i = [] ### after for loop, has same length as Vnew, shows the closest node that not in Vnew with Vnew.  
        max_inf = [] ### corresponding distance
        for i in range(len(Vnew)):  ## can be paralelled 
          vnew = Vnew[i]
          ListToSorted = [int(e) for e in M[:,vnew]]###
          index = sorted(range(len(ListToSorted)),key = lambda k: ListToSorted[k],reverse = True)
          index_i.append([ele for ele in index if ele not in Vnew][0]) 
          max_inf.append(M[index_i[-1],vnew])
      
        index1 = sorted(range(len(max_inf)),key = lambda k: max_inf[k],reverse = True)[0] ## relative position, Vnew[v1,v2] index_i[v4,v5] max_inf[s1,s2] index1 is the position in those 3 list
        Vnew.append(index_i[index1]) ### add in that node
        parent[index_i[index1]] = Vnew[index1] ## add direction, it has to be that the new added node is child, otherwise some nodes has 2 parents which is wrong.
      
      prim_time = time.process_time() - t
      return parent,mutual_inf_time,prim_time

    def fit(self,X,y,M = None):  ### this is based on trainning data !!!
      X, y = check_X_y(X, y)

      parent,mutual_inf_time,prim_time = self.Findparent(X,y)
      t = time.process_time()
      countDict = Counter(y)
      C = list(countDict.keys()) ### [class1 , class2, class3] in appearing order
      n,p = X.shape
      P_class = [(ele+self.alpha)/( n + self.alpha*len(C) )  for ele in list(countDict.values())]  ### prior for each class [p1,p2,p3], ### .values same order as .keys()
      P_class = dict(zip(C, P_class))  ## {c1:p1,c2:p2,c3:p3} ## should in correct order, .keys .values.
      Dict_C = {} ###  {c1:[counter1, ....counter8], c2:[counter1, ....counter8],   c3: [counter1, ....counter8]}
      K = {}

      root_i = self.starting_node ## 0 ,1 ,2 shows the position, thus int
      x_i = X[:,root_i]
      K[root_i] = len(Counter(x_i))
      for c in C: ### c origianl class label '1'   not 1
        ListCounter_c = {}
        row_inx_c = [row for row in range(n) if y[row] == c]
        x_i_c = X[row_inx_c,root_i]
        ListCounter_c[root_i] = Counter(x_i_c) ### list_counter_c keys are 0,1,2,3... showing position hence int. Counter(x_i_c) keys are original values of x, not position. hence not necesarily int
        for i in [e for e in range(0,p) if e != root_i]:
          if c == C[0]:
            x_i = X[:,i]
            K[i] =len(Counter(x_i))
          x_parent = X[:,parent[i]]
          x_parent_counter = Counter(x_parent)
          x_parent_counter_length = len(x_parent_counter)
          x_parent_value = list(x_parent_counter.keys())
          dict_i_c = {}
          for j in range(x_parent_counter_length):
            row_inx_c_parent_j = [row for row in range(n) if y[row] == c and x_parent[row] == x_parent_value[j]]
            x_i_c_p_j = X[row_inx_c_parent_j, i]
            dict_i_c[x_parent_value[j]] = Counter(x_i_c_p_j) ### x_parent_value[j] can make sure it is right key.
          ListCounter_c[i] = dict_i_c
        Dict_C[c] = ListCounter_c 

      CP_time = time.process_time() - t
      self.is_fitted_ = True 
      self.Dict_C_,self.p_,self.P_class_prior_,self.K_,self.classes_,self.countDict_, self.parent_ = Dict_C,p,P_class,K,np.array(C),countDict,parent
      self.training_time_ = np.array([mutual_inf_time,prim_time,CP_time])
      return self

    def predict_proba(self,X):	
      check_is_fitted(self)
      X = check_array(X)

      Prob_C = []
      root_i = self.starting_node

      for ins in X:
        P_class = self.P_class_prior_.copy()
        for c in self.classes_:
          ListCounter_c = self.Dict_C_[c]
          P_class[c] = P_class[c] * (ListCounter_c[root_i][ins[root_i]]+self.alpha) / (self.countDict_[c]+self.alpha*self.K_[root_i])
        
          for i in [e for e in range(0,self.p_) if e != root_i]:
            pValue = ins[self.parent_[i]] ### replicate C times
            try:###  ListCounter_c[i][pValue],pavlue does show in training
              Deno = sum(list(ListCounter_c[i][pValue].values() )) ## number of y =1, xparent = pvalue ,   ListCounter_c[i][pValue], pavlue does not show in training , keyerror
              P_class[c] = P_class[c] * (ListCounter_c[i][pValue][ins[i]] + self.alpha) / (Deno + self.alpha*self.K_[i]) ## ListCounter1[i][pValue][ins[i]] = number of y =1 xparent = pvalue, xi = xi
            except: ##ListCounter_c[i][pValue],pavlue does not show in training
              Deno = 0 ## ListCounter_c[i] this is when class == c, ith feature,  >> {parent(i) == value1: Counter,  parent(i) == value2: Counter  },  counter shows the distribution of x_i when class ==c and parent == pvalue
              P_class[c] = P_class[c] * (0 + self.alpha) / (Deno + self.alpha*self.K_[i])
        
        P_class = {key: P_class[key]/sum(list(P_class.values())) for key in P_class.keys()} ### normalize p_class
        Prob_C.append(list(P_class.values())) ### check the class order is correct

      Prob_C = np.array(Prob_C) ### for shap !!!!
      return Prob_C


In [40]:
tan = TAN()
tan.get_params()

{'alpha': 1, 'starting_node': 0}

In [41]:
tan.fit(X,y)
#tan.fit(X,y)
print(tan.predict_proba(X))
tan.score(X,y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:755: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  estimator=estimator)


[[0.86147555 0.13852445]
 [0.90809796 0.09190204]
 [0.92478606 0.07521394]
 ...
 [0.23448663 0.76551337]
 [0.21158355 0.78841645]
 [0.77417023 0.22582977]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).


0.8586387434554974

## STAN

In [0]:
class STAN(Bayes_net):
    name = "STAN"
    def __init__(self,alpha = 1,starting_node = 0):
      self.starting_node = starting_node
      self.alpha = alpha

    def Findparent(self,M):
      M = M.copy()
      np.fill_diagonal(M,0)  
      p = int(M.shape[0])  
      V = range(p)  #### . set of all nodes
      st = self.starting_node
      Vnew = [st] #### vertex that already found their parent. intitiate it with starting node. TAN randomly choose one
      parent = {st:None} ## use a dict to show nodes' interdepedency
      while set(Vnew) != set(V):   ### when their are still nodes whose parents are unknown.
        index_i = [] ### after for loop, has same length as Vnew, shows the closest node that not in Vnew with Vnew.  
        max_inf = [] ### corresponding distance
        for i in range(len(Vnew)):  ## can be paralelled 
          vnew = Vnew[i]
          ListToSorted = [int(e) for e in M[:,vnew]]###
          index = sorted(range(len(ListToSorted)),key = lambda k: ListToSorted[k],reverse = True)
          index_i.append([ele for ele in index if ele not in Vnew][0]) 
          max_inf.append(M[index_i[-1],vnew])
      
        index1 = sorted(range(len(max_inf)),key = lambda k: max_inf[k],reverse = True)[0] ## relative position, Vnew[v1,v2] index_i[v4,v5] max_inf[s1,s2] index1 is the position in those 3 list
        Vnew.append(index_i[index1]) ### add in that node
        parent[index_i[index1]] = Vnew[index1] ## add direction, it has to be that the new added node is child, otherwise some nodes has 2 parents which is wrong.
      
      return parent

    def fit(self,X,y,M):  ### this is based on trainning data !!!
      X, y = check_X_y(X, y)
      parent = self.Findparent(M)
      t = time.process_time()
      countDict = Counter(y)
      C = list(countDict.keys()) ### [class1 , class2, class3] in appearing order
      n,p = X.shape
      P_class = [(ele+self.alpha)/( n + self.alpha*len(C) )  for ele in list(countDict.values())]  ### prior for each class [p1,p2,p3], ### .values same order as .keys()
      P_class = dict(zip(C, P_class))
      Dict_C = {} ###  {c1:[counter1, ....counter8], c2:[counter1, ....counter8],   c3: [counter1, ....counter8]}
      K = {}

      root_i = self.starting_node ## 0 ,1 ,2 shows the position, thus int
      x_i = X[:,root_i]
      K[root_i] = len(Counter(x_i))
      for c in C: ### c origianl class label '1'   not 1
        ListCounter_c = {}
        row_inx_c = [row for row in range(n) if y[row] == c]
        x_i_c = X[row_inx_c,root_i]
        ListCounter_c[root_i] = Counter(x_i_c) ### list_counter_c keys are 0,1,2,3... showing position hence int. Counter(x_i_c) keys are original values of x, not position. hence not necesarily int
        for i in [e for e in range(0,p) if e != root_i]:
          if c == C[0]:
            x_i = X[:,i]
            K[i] =len(Counter(x_i))
          x_parent = X[:,parent[i]] ## will duplicate C times. 
          x_parent_counter = Counter(x_parent)
          x_parent_counter_length = len(x_parent_counter)
          x_parent_value = list(x_parent_counter.keys())
          dict_i_c = {}
          for j in range(x_parent_counter_length):
            row_inx_c_parent_j = [row for row in range(n) if y[row] == c and x_parent[row] == x_parent_value[j]]
            x_i_c_p_j = X[row_inx_c_parent_j, i]
            dict_i_c[x_parent_value[j]] = Counter(x_i_c_p_j) ### x_parent_value[j] can make sure it is right key.
          ListCounter_c[i] = dict_i_c
        Dict_C[c] = ListCounter_c 

      CP_time = np.array(time.process_time() - t)
      self.is_fitted_ = True
      self.Dict_C_,self.p_,self.P_class_prior_,self.K_,self.classes_,self.countDict_,self.parent_ = Dict_C,p,P_class,K,np.array(C),countDict,parent
      self.training_time_ = CP_time
      return self


    def predict_proba(self,X):	
      check_is_fitted(self)
      X = check_array(X)

      Prob_C = []
      root_i = self.starting_node

      for ins in X:
        P_class = self.P_class_prior_.copy()
        for c in self.classes_:
          ListCounter_c = self.Dict_C_[c]
          P_class[c] = P_class[c] * (ListCounter_c[root_i][ins[root_i]]+self.alpha) / (self.countDict_[c]+self.alpha*self.K_[root_i])
        
          for i in [e for e in range(0,self.p_) if e != root_i]:
            pValue = ins[self.parent_[i]] ### replicate C times
            try:###  ListCounter_c[i][pValue],pavlue does show in training
              Deno = sum(list(ListCounter_c[i][pValue].values() )) ## number of y =1, xparent = pvalue ,   ListCounter_c[i][pValue], pavlue does not show in training , keyerror
              P_class[c] = P_class[c] * (ListCounter_c[i][pValue][ins[i]] + self.alpha) / (Deno + self.alpha*self.K_[i]) ## ListCounter1[i][pValue][ins[i]] = number of y =1 xparent = pvalue, xi = xi
            except: ##ListCounter_c[i][pValue],pavlue does not show in training
              Deno = 0 ## ListCounter_c[i] this is when class == c, ith feature,  >> {parent(i) == value1: Counter,  parent(i) == value2: Counter  },  counter shows the distribution of x_i when class ==c and parent == pvalue
              P_class[c] = P_class[c] * (0 + self.alpha) / (Deno + self.alpha*self.K_[i])
        
        P_class = {key: P_class[key]/sum(list(P_class.values())) for key in P_class.keys()} ### normalize p_class
        Prob_C.append(list(P_class.values())) ### check the class order is correct

      Prob_C = np.array(Prob_C) ### for shap !!!!
      return Prob_C



In [18]:
stan = STAN()
stan.get_params()
stan.fit(X,y,M)
print(stan.predict_proba(X))
print(stan.name)
stan.score(X,y)

[[0.79215303 0.20784697]
 [0.91788404 0.08211596]
 [0.92852167 0.07147833]
 ...
 [0.24149027 0.75850973]
 [0.15284187 0.84715813]
 [0.74094713 0.25905287]]
STAN


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:755: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  estimator=estimator)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in sciki

0.8617801047120419

In [0]:
from sklearn.utils.estimator_checks import check_estimator

#check_estimator(NB)

## TAN_bagging

In [0]:
class TAN_bagging(Bayes_net):
  name = "TAN_bagging"
  def __init__(self, alpha = 1):
    self.alpha = alpha

  def fit(self,X,y,M = None): 
    """initialize model = [] . and training time."""
    X,y = check_X_y(X,y)
    n,p = X.shape ### number of features
    """fit base models"""
    training_time = 0
    models = []
    for i in range(p):
      model = TAN(self.alpha, starting_node= i)
      model.fit(X,y)
      models.append(model)
      training_time += model.training_time_

    self.models_ , self.p_= models,p
    self.training_time_ = training_time/p ### the fitting can be paralelled, hence define averge training time for this bagging
    self.is_fitted_ = True
    self.classes_ = model.classes_
    return self

  def predict_proba(self,X):	   
    check_is_fitted(self)
    X = check_array(X)

    Prob_C = 0
    for model in self.models_:
      Prob_C += model.predict_proba(X) ### get np array here 

    Prob_C = Prob_C/self.p_
    return(Prob_C)

  

In [0]:
tan_bag = TAN_bagging()
print(tan_bag.name)
tan_bag.fit(X,y)

In [44]:
tan_bag.predict_proba(X)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by usin

array([[0.82331053, 0.17668947],
       [0.90948728, 0.09051272],
       [0.93262318, 0.06737682],
       ...,
       [0.24045922, 0.75954078],
       [0.21006017, 0.78993983],
       [0.7522166 , 0.2477834 ]])

## STAN bagging

In [0]:
class STAN_bagging(Bayes_net):
  name = "STAN_bagging"
  def __init__(self,alpha = 1):
    self.alpha = alpha

  def fit(self,X,y,M): 
    X,y = check_X_y(X,y)
    n,p = X.shape
    training_time = 0
    models = []
    for i in range(p):
      model = STAN(self.alpha, starting_node= i)
      model.fit(X,y,M)
      models.append(model)
      training_time += model.training_time_

    self.models_, self.p_ = models,p
    self.training_time_ = training_time/p ### the fitting can be paralelled, hence define averge training time for this bagging
    self.is_fitted_ = True
    self.classes_ = model.classes_
    return self

  def predict_proba(self,X):	   
    check_is_fitted(self)
    X = check_array(X)

    Prob_C = 0
    for model in self.models_:
      Prob_C += model.predict_proba(X) ### get np array here 

    Prob_C = Prob_C/self.p_
    return(Prob_C)

  

In [0]:
stan_bag = STAN_bagging()
stan_bag.fit(X,y,M)
stan_bag.fit(X,y,M)

In [32]:
stan_bag.predict_proba(X)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using 

array([[0.7917778 , 0.2082222 ],
       [0.9187195 , 0.0812805 ],
       [0.92864436, 0.07135564],
       ...,
       [0.24276788, 0.75723212],
       [0.15375769, 0.84624231],
       [0.74263807, 0.25736193]])

## Ensemble TAN (STAN_TAN_bagging)

In [0]:
class STAN_TAN_bagging(Bayes_net):
  name = "STAN_TAN_bagging"
  def __init__(self,alpha = 1):
    self.alpha = alpha

  def fit(self,X,y,M): 
    X,y = check_X_y(X,y)
    n,p = X.shape  
    training_time = 0
    models = []
    ## train p TAN base models
    for i in range(p):
      model = TAN(self.alpha, starting_node= i)
      model.fit(X,y)
      models.append(model)
      training_time += model.training_time_

    #append STAN
    model = STAN(self.alpha, starting_node = 0) ### starting node not importance for TAN, very robust
    model.fit(X,y,M)
    models.append(model)    
    self.models_, self.p_ = models, p
    self.training_time_ = training_time/p ### after paralell, only consider average of p TAN_MT, ignore TAN since it takes less time than TAN_MT
    self.is_fitted_ = True
    self.classes_ = model.classes_
    return self

  def predict_proba(self,X):	   
    check_is_fitted(self)
    X = check_array(X)

    Prob_C = 0
    for model in self.models_:
      Prob_C += model.predict_proba(X) ### get np array here 

    Prob_C = Prob_C/(self.p_+ 1)
    return(Prob_C)

  

In [0]:
stan_tan_bag = STAN_TAN_bagging()
stan_tan_bag.fit(X,y,M)

In [36]:
stan_tan_bag.predict_proba(X)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:105: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by usin

array([[0.81984859, 0.18015141],
       [0.91042025, 0.08957975],
       [0.93216746, 0.06783254],
       ...,
       [0.24057378, 0.75942622],
       [0.20370258, 0.79629742],
       [0.75096444, 0.24903556]])

# Cross validation

In [0]:
import warnings
warnings.filterwarnings("ignore")
def get_cv(cls,X,Y,M,n_splits=10,cv_type = "KFold",verbose = True):  
  """ Cross validation to get CLL and accuracy and training time and precision and recall.
  """

  if cv_type == "StratifiedKFold":
    cv = StratifiedKFold(n_splits= n_splits, shuffle=True, random_state=42)##The folds are made by preserving the percentage of samples for each class.
  else: 
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
  

  model = cls()
  X,Y = check_X_y(X,Y)
  binarizer = MultiLabelBinarizer() ## for using recall and precision score
  binarizer.fit(Y)
  Accuracy = []
  Precision = []
  Recall = []
  CLL = []
  training_time = []
  for folder, (train_index, val_index) in enumerate(cv.split(X, Y)):#### X,Y are array, data is list
    X_train,X_val = X[train_index],X[val_index]
    y_train,y_val = Y[train_index],Y[val_index] 
    model.fit(X_train,y_train,M) ### whether data is list or array does not matter, only thing matters is label has to be same.
    training_time.append(model.training_time_)

    y_pred_prob= model.predict_proba(X_val)  
    y_pred_class = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred_class)
    precision = precision_score(binarizer.transform(y_val), 
         binarizer.transform(y_pred_class), 
         average='macro')    
    recall = recall_score(binarizer.transform(y_val), 
         binarizer.transform(y_pred_class), 
         average='macro')
    cll = model.Conditional_log_likelihood_general(y_val,y_pred_prob,model.classes_)
    if verbose:
        print("accuracy in %s fold is %s" % (folder+1,accuracy))
        print("CLL in %s fold is %s" % (folder+1,cll))
        print("precision in %s fold is %s" % (folder+1,precision))
        print("recall in %s fold is %s" % (folder+1,recall))
        print("training time in %s fold is %s" % (folder+1,training_time[-1]))
        print(10*'__')
    CLL.append(cll)
    Accuracy.append(accuracy)
    Recall.append(recall)
    Precision.append(precision)
  return Accuracy, CLL, training_time,Precision,Recall

In [66]:
Accuracy, CLL, training_time,Precision,Recall= get_cv(NB,X2,y2,M2)
print(np.mean(Accuracy))
print(np.mean(CLL))
print(np.mean(Precision))
print(np.mean(Recall))
print(np.mean(np.array(training_time)))

accuracy in 1 fold is 0.875
CLL in 1 fold is -26.97175334366014
precision in 1 fold is 0.9299999999999999
recall in 1 fold is 0.7307692307692308
training time in 1 fold is 0.00885077499999909
____________________
accuracy in 2 fold is 0.9107142857142857
CLL in 2 fold is -19.79193116126286
precision in 2 fold is 0.9509803921568627
recall in 2 fold is 0.75
training time in 2 fold is 0.005343936999999244
____________________
accuracy in 3 fold is 0.8181818181818182
CLL in 3 fold is -25.25170879090418
precision in 3 fold is 0.5563725490196079
recall in 3 fold is 0.5305851063829787
training time in 3 fold is 0.005570282000000759
____________________
accuracy in 4 fold is 0.9090909090909091
CLL in 4 fold is -19.86403033918379
precision in 4 fold is 0.7038690476190476
recall in 4 fold is 0.8357843137254901
training time in 4 fold is 0.006360927999999433
____________________
accuracy in 5 fold is 0.8909090909090909
CLL in 5 fold is -27.5560719322658
precision in 5 fold is 0.8259803921568627
re

In [67]:
Accuracy, CLL, training_time,Precision,Recall= get_cv(TAN,X2,y2,M2)
print(np.mean(Accuracy))
print(np.mean(CLL))
print(np.mean(Precision))
print(np.mean(Recall))
print(np.mean(np.array(training_time)))

accuracy in 1 fold is 0.9285714285714286
CLL in 1 fold is -14.530692699471452
precision in 1 fold is 0.9574468085106382
recall in 1 fold is 0.8461538461538461
training time in 1 fold is [3.47900804e-01 2.41824000e-04 1.54309450e-02]
____________________
accuracy in 2 fold is 0.9642857142857143
CLL in 2 fold is -11.353701614711582
precision in 2 fold is 0.9791666666666667
recall in 2 fold is 0.9
training time in 2 fold is [3.34512737e-01 2.44130000e-04 1.48101700e-02]
____________________
accuracy in 3 fold is 0.8181818181818182
CLL in 3 fold is -32.343225944163294
precision in 3 fold is 0.6555555555555556
recall in 3 fold is 0.6861702127659575
training time in 3 fold is [3.40365407e-01 2.61194000e-04 1.49456560e-02]
____________________
accuracy in 4 fold is 0.9272727272727272
CLL in 4 fold is -15.107186188084356
precision in 4 fold is 0.75
recall in 4 fold is 0.9607843137254901
training time in 4 fold is [3.45646008e-01 2.56731000e-04 1.56065940e-02]
____________________
accuracy in 5

In [68]:
Accuracy, CLL, training_time,Precision,Recall= get_cv(STAN,X2,y2,M2)
print(np.mean(Accuracy))
print(np.mean(CLL))
print(np.mean(Precision))
print(np.mean(Recall))
print(np.mean(np.array(training_time)))

accuracy in 1 fold is 0.9642857142857143
CLL in 1 fold is -15.904356789215896
precision in 1 fold is 0.9777777777777779
recall in 1 fold is 0.9230769230769231
training time in 1 fold is 0.020265178000002493
____________________
accuracy in 2 fold is 0.9464285714285714
CLL in 2 fold is -11.995505522810125
precision in 2 fold is 0.9231678486997636
recall in 2 fold is 0.8891304347826088
training time in 2 fold is 0.015218664999999021
____________________
accuracy in 3 fold is 0.8545454545454545
CLL in 3 fold is -25.68123966870797
precision in 3 fold is 0.7074468085106382
recall in 3 fold is 0.7074468085106382
training time in 3 fold is 0.014232937999999251
____________________
accuracy in 4 fold is 0.8909090909090909
CLL in 4 fold is -15.979338550197971
precision in 4 fold is 0.7
recall in 4 fold is 0.9411764705882353
training time in 4 fold is 0.014199173000001508
____________________
accuracy in 5 fold is 0.9454545454545454
CLL in 5 fold is -16.856372481252627
precision in 5 fold is 0.9

In [69]:
Accuracy, CLL, training_time,Precision,Recall= get_cv(TAN_bagging,X2,y2,M2)
print(np.mean(Accuracy))
print(np.mean(CLL))
print(np.mean(Precision))
print(np.mean(Recall))
print(np.mean(np.array(training_time)))

accuracy in 1 fold is 0.9285714285714286
CLL in 1 fold is -14.7988999751734
precision in 1 fold is 0.9574468085106382
recall in 1 fold is 0.8461538461538461
training time in 1 fold is [3.27753114e-01 2.50680000e-04 1.51104425e-02]
____________________
accuracy in 2 fold is 0.9642857142857143
CLL in 2 fold is -11.770376562035882
precision in 2 fold is 0.9791666666666667
recall in 2 fold is 0.9
training time in 2 fold is [3.32170815e-01 2.69688000e-04 1.53214486e-02]
____________________
accuracy in 3 fold is 0.8545454545454545
CLL in 3 fold is -30.611296032426164
precision in 3 fold is 0.7074468085106382
recall in 3 fold is 0.7074468085106382
training time in 3 fold is [3.19645309e-01 2.44505625e-04 1.47279760e-02]
____________________
accuracy in 4 fold is 0.9272727272727272
CLL in 4 fold is -15.55060427090059
precision in 4 fold is 0.75
recall in 4 fold is 0.9607843137254901
training time in 4 fold is [3.31462835e-01 2.62280375e-04 1.67925664e-02]
____________________
accuracy in 5 fo

In [70]:
Accuracy, CLL, training_time,Precision,Recall= get_cv(STAN_bagging,X2,y2,M2)
print(np.mean(Accuracy))
print(np.mean(CLL))
print(np.mean(Precision))
print(np.mean(Recall))
print(np.mean(np.array(training_time)))

accuracy in 1 fold is 0.9642857142857143
CLL in 1 fold is -15.944546226460112
precision in 1 fold is 0.9777777777777779
recall in 1 fold is 0.9230769230769231
training time in 1 fold is 0.01824390912500018
____________________
accuracy in 2 fold is 0.9464285714285714
CLL in 2 fold is -11.979787931931426
precision in 2 fold is 0.9231678486997636
recall in 2 fold is 0.8891304347826088
training time in 2 fold is 0.015136318500000634
____________________
accuracy in 3 fold is 0.8545454545454545
CLL in 3 fold is -25.600493196824697
precision in 3 fold is 0.7074468085106382
recall in 3 fold is 0.7074468085106382
training time in 3 fold is 0.0168353346249992
____________________
accuracy in 4 fold is 0.8909090909090909
CLL in 4 fold is -15.930624895994418
precision in 4 fold is 0.7
recall in 4 fold is 0.9411764705882353
training time in 4 fold is 0.015397226250000173
____________________
accuracy in 5 fold is 0.9454545454545454
CLL in 5 fold is -16.722488286379274
precision in 5 fold is 0.907

In [71]:
Accuracy, CLL, training_time,Precision,Recall= get_cv(STAN_TAN_bagging,X2,y2,M2)
print(np.mean(Accuracy))
print(np.mean(CLL))
print(np.mean(Precision))
print(np.mean(Recall))
print(np.mean(np.array(training_time)))

accuracy in 1 fold is 0.9464285714285714
CLL in 1 fold is -14.734132799884247
precision in 1 fold is 0.9673913043478262
recall in 1 fold is 0.8846153846153846
training time in 1 fold is [3.26308282e-01 2.55383125e-04 1.52829092e-02]
____________________
accuracy in 2 fold is 0.9821428571428571
CLL in 2 fold is -11.709841765142745
precision in 2 fold is 0.9893617021276595
recall in 2 fold is 0.95
training time in 2 fold is [3.27309937e-01 2.67105375e-04 1.57623266e-02]
____________________
accuracy in 3 fold is 0.8727272727272727
CLL in 3 fold is -29.684117150877217
precision in 3 fold is 0.7440476190476191
recall in 3 fold is 0.7180851063829787
training time in 3 fold is [3.27277832e-01 2.64283625e-04 1.50628981e-02]
____________________
accuracy in 4 fold is 0.9272727272727272
CLL in 4 fold is -15.479929971141784
precision in 4 fold is 0.75
recall in 4 fold is 0.9607843137254901
training time in 4 fold is [3.29968530e-01 2.65865625e-04 1.51590938e-02]
____________________
accuracy in 